In [3]:
import numpy as np
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
def str_to_set(g):
    return set(item['name'] for item in eval(g))

meta = pd.read_csv('/content/gdrive/MyDrive/the-movies-dataset/movies_metadata.csv')

meta = meta[['id', 'original_title', 'original_language', 'genres']]

meta = meta.rename(columns={'id': 'movieId', 'original_title': 'title', 'original_language': 'language'})

meta = meta.loc[meta['language'] == 'en', :]

meta.movieId = pd.to_numeric(meta.movieId)

meta.genres = meta.genres.apply(str_to_set)

meta

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,movieId,title,language,genres
0,862,Toy Story,en,"{Family, Animation, Comedy}"
1,8844,Jumanji,en,"{Adventure, Family, Fantasy}"
2,15602,Grumpier Old Men,en,"{Romance, Comedy}"
3,31357,Waiting to Exhale,en,"{Romance, Drama, Comedy}"
4,11862,Father of the Bride Part II,en,{Comedy}
...,...,...,...,...
45459,222848,Caged Heat 3000,en,{Science Fiction}
45460,30840,Robin Hood,en,"{Romance, Drama, Action}"
45463,67758,Betrayal,en,"{Drama, Action, Thriller}"
45464,227506,Satana likuyushchiy,en,{}


In [6]:
keywords = pd.read_csv('/content/gdrive/MyDrive/the-movies-dataset/keywords.csv')
keywords = keywords.rename(columns={'id':'movieId'})

keywords.keywords = keywords.keywords.apply(str_to_set)

keywords

,movieId,keywords
0,862,"{toy, rivalry, jealousy, friends, boy next doo..."
1,8844,"{disappearance, recluse, giant insect, based o..."
2,15602,"{old men, best friend, duringcreditsstinger, f..."
3,31357,"{divorce, chick flick, interracial relationshi..."
4,11862,"{midlife crisis, baby, pregnancy, aging, confi..."
...,...,...
46414,439050,{tragic love}
46415,111109,"{pinoy, artist, play}"
46416,67758,{}
46417,227506,{}


In [7]:
merged = pd.merge(meta, keywords, on='movieId', how='inner')
merged

,movieId,title,language,genres,keywords
0,862,Toy Story,en,"{Family, Animation, Comedy}","{toy, rivalry, jealousy, friends, boy next doo..."
1,8844,Jumanji,en,"{Adventure, Family, Fantasy}","{disappearance, recluse, giant insect, based o..."
2,15602,Grumpier Old Men,en,"{Romance, Comedy}","{old men, best friend, duringcreditsstinger, f..."
3,31357,Waiting to Exhale,en,"{Romance, Drama, Comedy}","{divorce, chick flick, interracial relationshi..."
4,11862,Father of the Bride Part II,en,{Comedy},"{midlife crisis, baby, pregnancy, aging, confi..."
...,...,...,...,...,...
32847,222848,Caged Heat 3000,en,{Science Fiction},{}
32848,30840,Robin Hood,en,"{Romance, Drama, Action}",{}
32849,67758,Betrayal,en,"{Drama, Action, Thriller}",{}
32850,227506,Satana likuyushchiy,en,{},{}


In [8]:
dk = merged.loc[merged.title == 'The Dark Knight', :].iloc[0, :]
dkr = merged.loc[merged.title == 'The Dark Knight Rises', :].iloc[0, :]

pd.concat([dk, dkr], axis=1).T

,movieId,title,language,genres,keywords
10278,155,The Dark Knight,en,"{Crime, Drama, Action, Thriller}","{imax, tragic hero, based on comic, organized ..."
14315,49026,The Dark Knight Rises,en,"{Crime, Drama, Action, Thriller}","{destruction, imax, hostage drama, time bomb, ..."


In [9]:
def jaccard_similarity(s1, s2):
    if len(s1 | s2) ==0:
        return 0
    return len(s1 & s2) / len(s1 | s2)

dk_set = dk.genres | dk.keywords
dkr_set = dkr.genres | dkr.keywords

jaccard_similarity(dk_set, dkr_set)

0.37142857142857144

In [12]:
ratings = pd.read_csv("/content/gdrive/MyDrive/the-movies-dataset/ratings_small.csv")
ratings = pd.merge(ratings, meta[['movieId', 'title']], on='movieId', how='inner')
ratings

,userId,movieId,rating,timestamp,title
0,1,1371,2.5,1260759135,Rocky III
1,4,1371,4.0,949810302,Rocky III
2,7,1371,3.0,851869160,Rocky III
3,19,1371,4.0,855193404,Rocky III
4,21,1371,3.0,853852263,Rocky III
...,...,...,...,...,...
33276,652,127728,5.0,1439586990,8:46
33277,652,129009,4.0,1442690827,Love Is a Ball
33278,653,2103,3.0,948161066,Solaris
33279,659,167,4.0,836137550,K-PAX


In [13]:
matrix = ratings.pivot_table(index='userId', columns='title', values='rating')
matrix

title,!Women Art Revolution,'Gator Bait,'Twas the Night Before Christmas,10 Items or Less,10 Things I Hate About You,"10,000 BC",11'09''01 - September 11,12 + 1,12 Angry Men,1408,15 Minutes,16 Blocks,1984,2 Days in Paris,"20,000 Leagues Under the Sea",2001: A Space Odyssey,2010,2061 - Un anno eccezionale,21 Grams,24 Hour Party People,25th Hour,28 Days Later,28 Weeks Later,29th Street,2:37,3 Ninjas: High Noon at Mega Mountain,300,40 Days and 40 Nights,42nd Street,48 Hrs.,4: Rise of the Silver Surfer,5 Card Stud,5 Days of War,50 First Dates,54,8 Mile,88 Minutes,8:46,9 Songs,95 Miles to Go,...,Wild Things,Wild Tigers I Have Known,Wild West Comedy Show: 30 Days & 30 Nights - Hollywood to the Heartland,Wild Wild West,Wild at Heart,Wild in the Country,Will Penny,Willy Wonka & the Chocolate Factory,Wish You Were Here,Within the Woods,Without You I'm Nothing,Wizards of Waverly Place: The Movie,Wizards of the Lost Kingdom,Wolves of Wall Street,Working Girl,World Trade Center,Wuthering Heights,X-Men Origins: Wolverine,X: The Unknown,Yamakasi - Les samouraïs des temps modernes,Yankee Doodle Dandy,You Are Not I,You Instead,You Only Live Twice,You're a Big Boy Now,"You, Me and Dupree",Young Adam,Young Black Stallion,Young Frankenstein,Young Mr. Lincoln,Young and Innocent,Zaat,Zabriskie Point,Zapped Again!,Zardoz,Zodiac,eXistenZ,xXx,¡Three Amigos!,Мой сводный брат Франкенштейн
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [14]:
def pearson_similarity(u1, u2):
    u1 = u1 - u1.mean()
    u2 = u2 - u2.mean()
    denom = ((u1 ** 2).sum() ** 0.5) * ((u2 ** 2).sum() ** 0.5)
    if denom != 0:
        return (u1 * u2).sum() / denom
    else:
        return 0

In [15]:
dk_rating = matrix.loc[:, 'The Dark Knight']
pk_rating = matrix.loc[:, 'Prom Night']


pearson_similarity(dk_rating, pk_rating)

0.7735659346940948

In [16]:
def find_similar_movies(input_title, n, alpha, matrix, meta):
    input_vec = matrix.loc[:, input_title]
    
    input_meta = merged.loc[merged['title'] == input_title, :].iloc[0,:]
    input_set = input_meta['genres'] | input_meta['keywords']
    
    result = []
    pearson_scores = []
     
    for this_title in matrix.columns:
        this_vec = matrix.loc[:, this_title]
            
        this_meta = merged.loc[merged['title'] == this_title, :].iloc[0,:]
        this_set = this_meta['genres'] | this_meta['keywords']
        
        pearson_score = pearson_similarity(input_vec, this_vec)
        pearson_scores.append(pearson_score)
        
        jaccard_score = jaccard_similarity(input_set, this_set)
        
        score = alpha * pearson_score + (1-alpha) * jaccard_score
        result.append((this_title, jaccard_score, pearson_score, score))
        
    result.sort(key=lambda r: r[3], reverse=True)
    
    return result[:n]

In [17]:
top20 = find_similar_movies('Prom Night', 20, 0.3, matrix, meta)
pd.DataFrame(top20, columns = ['title', 'jaccard', 'pearson', 'score'])

,title,jaccard,pearson,score
0,Prom Night,1.000000,1.000000,1.000000
1,Wild Wild West,0.000000,1.000000,0.300000
2,The Dark Knight,0.022222,0.773566,0.247625
3,Population 436,0.103448,0.577350,0.245619
4,Midnight in the Garden of Good and Evil,0.071429,0.572293,0.221688
5,Freddy vs. Jason,0.166667,0.316228,0.211535
6,Wet Hot American Summer,0.038462,0.612372,0.210635
7,The Sentinel,0.066667,0.545545,0.210330
8,Godzilla,0.025641,0.632456,0.207685
9,Deadlier Than the Male,0.037037,0.577350,0.199131


In [18]:
result = find_similar_movies('The Dark Knight', 10, 0.3, matrix, meta)
pd.DataFrame(result, columns = ['title', 'jaccard', 'pearson', 'score'])

,title,jaccard,pearson,score
0,The Dark Knight,1.000000,1.000000,1.000000
1,Wild Wild West,0.032258,0.773566,0.254650
2,Prom Night,0.022222,0.773566,0.247625
3,Batman Begins,0.292683,0.005015,0.206383
4,Yamakasi - Les samouraïs des temps modernes,0.125000,0.377145,0.200643
5,Blue Thunder,0.133333,0.326617,0.191318
6,Midnight in the Garden of Good and Evil,0.111111,0.373841,0.189930
7,Topaz,0.103448,0.377145,0.185557
8,Big Bad Mama,0.107143,0.344649,0.178395
9,Sneakers,0.068966,0.415830,0.173025
